In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_ucl_season(url):
    html=requests.get(url).text
    soup=BeautifulSoup(html,"html.parser")

    matches=[]

    table=soup.find("table",{"id":"sched_8_2023-2024_8"}) or \
          soup.find("table",{"id":"sched_8_2023-2024_8"}) or \
          soup.find('table')

    for row in table.find_all('tr'):
        cols=row.find_all('td')


        if len(cols)<5:
            continue


        date=cols[0].text.strip()
        home=cols[2].text.strip()
        score=cols[3].text.strip()
        away=cols[4].text.strip()
        

        if score =="":
            continue

        matches.append({
            "date":date,
            "home_team":home,
            "away_team":away,
            "score":score
        })

        return pd.Dataframe(matches)
    
ucl_2023= "https://fbref.com/en/comps/8/2023-2024/schedule/2023-2024-Champions-League-Scores-and-Fixtures"
ucl_2024 = "https://fbref.com/en/comps/8/2024-2025/schedule/2024-2025-Champions-League-Scores-and-Fixtures"  

df_2023=scrape_ucl_season(ucl_2023)

df_2023.head()

AttributeError: 'NoneType' object has no attribute 'find_all'